In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pymorphy3
import re
from tqdm import tqdm
import umap

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier

# Разложение матрицы на три с сокращением размерности.
from sklearn.decomposition import TruncatedSVD

In [5]:
news_texts = pd.DataFrame(pd.read_csv("hrefs.CSV"), columns=['title', 'text'])
news_texts

,title,text
0,"В МИД Японии заявили, что страна предоставила ...",\nВласти Японии к настоящему времени предостав...
1,Белгородская область подверглась новой атаке б...,"\nНочью в среду, 24 мая, в Белгородской област..."
2,Климатолог рассказал о сезоне ураганов в Москве,\nВ столичном регионе в конце мая – начале июн...
3,Российский холдинг Segezha Group продал европе...,\nРоссийский лесопромышленный холдинг Segezha ...
4,"Жители Москвы могут выбрать цвета, которыми об...",\nЖители Москвы могут проголосовать за одну из...
...,...,...
403,Лидеры G7 договорились продолжить поддержку Ук...,\nПравительство России расширило доступ произв...
404,"В кабмине ожидают, что рынок БПЛА в России по ...","\nЛидеры государств, входящих в ""Большую семер..."
405,РИА Новости: в России средний размер автокреди...,\nNikkei: Япония запретит предоставлять России...
406,ВСУ обстреляли село Муром Белгородской области,\nИтальянский ракетный эскадренный миноносец п...


In [7]:
%%time
# Считаем матрицу термин-документ, но не с частотами, а со значениями Tf*Idf
X = TfidfVectorizer().fit_transform(news_texts['text'])
# Проводим SVD-разложение по 20 компонентам.
svd = TruncatedSVD(n_components=20)
X2 = svd.fit_transform(X)

explained_variance = svd.explained_variance_ratio_.sum()
print(f"Explained variance of the SVD step: {int(explained_variance * 100)}%")
# Просто чтобы посмотреть, что там в самом деле вектора.
print(X2.shape, X2)

Explained variance of the SVD step: 14%
(408, 20) [[ 0.19211723 -0.05983716 -0.05851599 ...  0.05303783  0.06798314
   0.11400188]
 [ 0.26220213  0.29105885 -0.03566937 ... -0.03496959 -0.02840279
   0.03871183]
 [ 0.15600969 -0.00441021  0.23660962 ... -0.06328228 -0.01617342
   0.05322759]
 ...
 [ 0.22544302 -0.01240082  0.04857593 ...  0.04296395 -0.03003133
  -0.02411102]
 [ 0.16817652 -0.0306237  -0.01669047 ... -0.00448548  0.02142385
  -0.07626606]
 [ 0.11752255  0.01529366  0.03035909 ... -0.02441485 -0.01273865
   0.00630432]]
CPU times: total: 312 ms
Wall time: 131 ms


In [9]:
X

<408x16600 sparse matrix of type '<class 'numpy.float64'>'
	with 56024 stored elements in Compressed Sparse Row format>

In [12]:
# Список значимых частей речи. 
# Они нам потом понадобятся в немного другом виде. Так что сделаем словарь. чтобы два раза не вставать.
conv_pos = {'ADJF':'ADJ', 'ADJS':'ADJ', 'ADV':'ADV', 'NOUN':'NOUN', 
            'VERB':'VERB', 'PRTF':'ADJ', 'PRTS':'ADJ', 'GRND':'VERB'}

tmp_dict = {} # Кеш значимых слов.
nones = {} # Кеш незначимых слов.

morph = pymorphy3.MorphAnalyzer()

# Фильтруем по части речи и возвращаем только начальную форму.
def normalizePymorphy(text, need_pos=True):
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    words = []
    for t in tokens:
        # Если токен уже был закеширован, быстро возьмем результат из него.
        if t in tmp_dict.keys():
            words.append(tmp_dict[t])
        # Аналогично, если он в кеше незначимых слов.
        elif t in nones.keys():
            pass
        # Слово еще не встретилось, будем проводить медленный морфологический анализ.
        else:
            pv = morph.parse(t)
            if pv[0].tag.POS != None:
                if pv[0].tag.POS in conv_pos.keys():
                    if need_pos:
                        word = pv[0].normal_form+"_"+conv_pos[pv[0].tag.POS]
                    else:
                        word = pv[0].normal_form
                    # Отправляем слово в результат, ...
                    words.append(word)
                    # ... и кешируем результат его разбора.
                    tmp_dict[t] = word
                else:
                    # Для незначимых слов можно даже ничего не хранить. Лишь бы потом не обращаться к морфологии.
                    nones[t] = ""
                    
    return words

In [15]:
%%time
# Приведем слова в текстах к начальным формам.
news_texts['NText'] = news_texts['text'].map(lambda x:' '.join(normalizePymorphy(x)))

CPU times: total: 1.34 s
Wall time: 1.38 s


In [17]:

%%time
X_l = TfidfVectorizer().fit_transform(news_texts['NText'])
svd = TruncatedSVD(n_components=10)
X2_l = svd.fit_transform(X)

CPU times: total: 78.1 ms
Wall time: 85 ms


In [18]:
# Подготовим целевые переменные. Мы же знаем, что там ровно по 1000 текстов для каждой области.
classes = np.ones(5000)
for i in range(2, 6):
    classes[(i-1)*1000: i*1000] = i